# How to handle transactions and concurrency

Handling transactions and concurrency with C# and Dapper is an important aspect of developing reliable and robust data access layers for your applications. Transactions allow you to group multiple operations into a single unit of work, and ensure that either all of them succeed or none of them do. Concurrency refers to the situation where multiple users or processes try to access or modify the same data at the same time, and how to resolve any conflicts or inconsistencies that may arise.

To handle transactions and concurrency with C# and Dapper, you can use the following tips and best practices:

* Use the <code>IDbConnection.BeginTransaction</code> method to start a new transaction, and use the <code>IDbTransaction.Commit</code> or <code>IDbTransaction.Rollback</code> methods to complete or cancel the transaction. You can pass the <code>IDbTransaction</code> object as a parameter to the Dapper methods, such as <code>Execute</code>, <code>Query</code>, or <code>QueryAsync</code>, to execute the operations within the transaction scope.
* Use the <code>IsolationLevel</code> enumeration to specify the level of isolation for your transaction, which determines how the transaction interacts with other concurrent transactions. The higher the isolation level, the more locks are taken on the data, and the less concurrency issues occur, but the more performance overhead is incurred. The default isolation level for SQL Server is Read Committed, which means that the transaction cannot read data that has been modified but not committed by other transactions.
* Use the <code>rowversion</code> data type to automatically generate a binary number that indicates the version of a row in a table. You can use this number to detect any changes made to the row by other transactions, and handle the concurrency conflicts accordingly. For example, you can use the rowversion value to compare the original and current values of the row, and decide whether to overwrite, merge, or reject the changes.
* Use the <code>optimistic</code> or <code>pessimistic</code> concurrency control strategies to handle concurrency conflicts. Optimistic concurrency assumes that conflicts are rare, and allows multiple users to read and modify the same data without locking it. However, if a conflict occurs, the transaction that tries to commit the changes last will fail, and will have to retry or abort the operation. Pessimistic concurrency assumes that conflicts are frequent, and locks the data when a user reads or modifies it, preventing other users from accessing it until the transaction is completed. This avoids conflicts, but reduces the concurrency and performance of the system.

To illustrate these tips and best practices, let’s take a look at an example of how to handle transactions and concurrency with C# and Dapper using the AdventureWorks2022 database. The AdventureWorks2022 database is a sample database that contains data about a fictional company that sells bicycles and related products. You can download the database from here.

Let’s assume that we want to create a data access layer for the Product table, which contains information about the products that the company sells. The first step to handle transactions and concurrency with C# and Dapper is to create a C# class that represents the Product entity. This class will map the columns of the Product table to its properties. We can use the following code to create the Product class:


In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:Dapper"
using Microsoft.Data.SqlClient;
using System;

// A class that represents the Product entity
public class Product
{
    public int ProductID { get; set; }
    public string Name { get; set; }
    public string ProductNumber { get; set; }
    public string Color { get; set; }
    public decimal StandardCost { get; set; }
    public decimal ListPrice { get; set; }
    public string Size { get; set; }
    public decimal? Weight { get; set; }
    public int? ProductCategoryID { get; set; }
    public int? ProductModelID { get; set; }
    public DateTime SellStartDate { get; set; }
    public DateTime? SellEndDate { get; set; }
    public DateTime? DiscontinuedDate { get; set; }
    public Guid rowguid { get; set; }
    public DateTime ModifiedDate { get; set; }
    public byte[] RowVersion { get; set; }
}

Installed Packages Dapper, 2.1.28 Microsoft.Data.SqlClient, 5.1.2

Notice how we use the <code>RowVersion</code> property to store the rowversion value of the row, which will help us detect any concurrency conflicts.

The next step is to create an interface that defines the contract for the data access operations that we want to perform on the Product table. This interface will abstract the implementation details of the data access layer and will allow us to use dependency injection and unit testing. We can use the following code to create the IProductRepository interface:


In [3]:
using System.Collections.Generic;
using System.Threading.Tasks;

// An interface that defines the contract for the data access operations on the Product table
public interface IProductRepository
{    
    Task<IEnumerable<Product>> GetAllProductsAsync();
    Task<Product> GetProductByIdAsync(int productId);
    Task<IEnumerable<Product>> GetProductsByCategoryIdAsync(int categoryId);
    Task<IEnumerable<Product>> GetProductsByModelIdAsync(int modelId);
    Task<int> AddProductAsync(Product product);
    Task<int> UpdateProductAsync(Product product);
    Task<int> DeleteProductAsync(int productId);
}

Notice how we use the <code>async</code> and <code>await</code> keywords to perform asynchronous data access operations, and we use the <code>Task&lt;T&gt;</code> type to return the results of the operations.

The final step is to create a class that implements the <code>IProductRepository</code> interface and uses Dapper to execute the SQL queries and commands on the Product table. This class will contain the actual data access logic and will use Dapper’s extension methods, query builders, and custom mappers to simplify the code and reduce boilerplate code. We can use the following code to create the ProductRepository class:

In [4]:
using System.Collections.Generic;
using System.Data;
using System.Data.SqlClient;
using System.Linq;
using System.Threading.Tasks;
using Dapper;

// A class that implements the IProductRepository interface and uses Dapper to execute the SQL queries and commands on the Product table
public class ProductRepository : IProductRepository
{
    // The connection string to the AdventureWorks2022 database
    private readonly string _connectionString;

    // The constructor that takes the connection string as a parameter
    public ProductRepository(string connectionString)
    {
        _connectionString = connectionString;
    }

    // A method that returns all the products
    public async Task<IEnumerable<Product>> GetAllProductsAsync()
    {
        // Create a SQL query to select all the products
        var sql = @"SELECT * FROM Production.Product";

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {    
            // Execute the query and return the results as a list of Product objects
            return await connection.QueryAsync<Product>(sql);
        }
    }

    // A method that returns a product by its ID
    public async Task<Product> GetProductByIdAsync(int productId)
    {
        // Create a SQL query to select a product by its ID
        var sql = @"SELECT * FROM Production.Product WHERE ProductID = @ProductID";

        // Create a dynamic parameter object with the product ID
        var parameters = new DynamicParameters();
        parameters.Add("@ProductID", productId);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {            
            // Execute the query and return the result as a Product object
            return await connection.QueryFirstOrDefaultAsync<Product>(sql, parameters);
        }
    }

    // A method that returns the products by their category ID
    public async Task<IEnumerable<Product>> GetProductsByCategoryIdAsync(int categoryId)
    {
        // Create a SQL query to select the products by their category ID
        var sql = @"SELECT p.* FROM Production.Product p
                    INNER JOIN Production.ProductCategory pc
                    ON p.ProductCategoryID = pc.ProductCategoryID
                    WHERE pc.ProductCategoryID = @ProductCategoryID";

        // Create a dynamic parameter object with the category ID
        var parameters = new DynamicParameters();
        parameters.Add("@ProductCategoryID", categoryId);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {         
            // Execute the query and return the results as a list of Product objects
            return await connection.QueryAsync<Product>(sql, parameters);
        }
    }

    // A method that returns the products by their model ID
    public async Task<IEnumerable<Product>> GetProductsByModelIdAsync(int modelId)
    {
        // Create a SQL query to select the products by their model ID
        var sql = @"SELECT p.* FROM Production.Product p
                    INNER JOIN Production.ProductModel pm
                    ON p.ProductModelID = pm.ProductModelID
                    WHERE pm.ProductModelID = @ProductModelID";

        // Create a dynamic parameter object with the model ID
        var parameters = new DynamicParameters();
        parameters.Add("@ProductModelID", modelId);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {           
            // Execute the query and return the results as a list of Product objects
            return await connection.QueryAsync<Product>(sql, parameters);
        }
    }

    // A method that adds a new product
    public async Task<int> AddProductAsync(Product product)
    {
        // Create a SQL command to insert a new product
        var sql = @"INSERT INTO Production.Product
                    (Name, ProductNumber, Color, StandardCost, ListPrice, Size, Weight, ProductCategoryID, ProductModelID, SellStartDate, SellEndDate, DiscontinuedDate, rowguid, ModifiedDate, RowVersion)
                    VALUES
                    (@Name, @ProductNumber, @Color, @StandardCost, @ListPrice, @Size, @Weight, @ProductCategoryID, @ProductModelID, @SellStartDate, @SellEndDate, @DiscontinuedDate, @rowguid, @ModifiedDate, DEFAULT);
                    SELECT SCOPE_IDENTITY();";

        // Create a dynamic parameter object with the product properties
        var parameters = new DynamicParameters();
        parameters.Add("@Name", product.Name);
        parameters.Add("@ProductNumber", product.ProductNumber);
        parameters.Add("@Color", product.Color);
        parameters.Add("@StandardCost", product.StandardCost);
        parameters.Add("@ListPrice", product.ListPrice);
        parameters.Add("@Size", product.Size);
        parameters.Add("@Weight", product.Weight);
        parameters.Add("@ProductCategoryID", product.ProductCategoryID);
        parameters.Add("@ProductModelID", product.ProductModelID);
        parameters.Add("@SellStartDate", product.SellStartDate);
        parameters.Add("@SellEndDate", product.SellEndDate);
        parameters.Add("@DiscontinuedDate", product.DiscontinuedDate);
        parameters.Add("@rowguid", product.rowguid);
        parameters.Add("@ModifiedDate", product.ModifiedDate);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {           
            // Execute the command and return the result as the new product ID
            return await connection.ExecuteScalarAsync<int>(sql, parameters);
        }
    }

    // A method that updates an existing product
    public async Task<int> UpdateProductAsync(Product product)
    {
        // Create a SQL command to update an existing product
        var sql = @"UPDATE Production.Product
                    SET Name = @Name,
                    ProductNumber = @ProductNumber,
                    Color = @Color,
                    StandardCost = @StandardCost,
                    ListPrice = @ListPrice,
                    Size = @Size,
                    Weight = @Weight,
                    ProductCategoryID = @ProductCategoryID,
                    ProductModelID = @ProductModelID,
                    SellStartDate = @SellStartDate,
                    SellEndDate = @SellEndDate,
                    DiscontinuedDate = @DiscontinuedDate,
                    rowguid = @rowguid,
                    ModifiedDate = @ModifiedDate
                    WHERE ProductID = @ProductID AND RowVersion = @RowVersion";

        // Create a dynamic parameter object with the product properties
        var parameters = new DynamicParameters();
        parameters.Add("@ProductID", product.ProductID);
        parameters.Add("@Name", product.Name);
        parameters.Add("@ProductNumber", product.ProductNumber);
        parameters.Add("@Color", product.Color);
        parameters.Add("@StandardCost", product.StandardCost);
        parameters.Add("@ListPrice", product.ListPrice);
        parameters.Add("@Size", product.Size);
        parameters.Add("@Weight", product.Weight);
        parameters.Add("@ProductCategoryID", product.ProductCategoryID);
        parameters.Add("@ProductModelID", product.ProductModelID);
        parameters.Add("@SellStartDate", product.SellStartDate);
        parameters.Add("@SellEndDate", product.SellEndDate);
        parameters.Add("@DiscontinuedDate", product.DiscontinuedDate);
        parameters.Add("@rowguid", product.rowguid);
        parameters.Add("@ModifiedDate", product.ModifiedDate);
        parameters.Add("@RowVersion", product.RowVersion);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {           
            // Execute the command and return the result as the number of affected rows
            return await connection.ExecuteAsync(sql, parameters);
        }
    }

    // A method that deletes an existing product
    public async Task<int> DeleteProductAsync(int productId)
    {
        // Create a SQL command to delete an existing product
        var sql = @"DELETE FROM Production.Product WHERE ProductID = @ProductID";

        // Create a dynamic parameter object with the product ID
        var parameters = new DynamicParameters();
        parameters.Add("@ProductID", productId);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {           
            // Execute the command and return the result as the number of affected rows
            return await connection.ExecuteAsync(sql, parameters);
        }
    }
}

Notice how we use the <code>RowVersion</code> property to check for concurrency conflicts in the <code>UpdateProductAsync</code> method. We use the <code>WHERE</code> clause to compare the <code>RowVersion</code> value of the product with the value stored in the database. If they are not equal, it means that another transaction has modified the product, and the update will fail. We can handle this exception in the calling code and notify the user accordingly.